In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('seaborn-whitegrid')
%matplotlib inline
import pickle
from collections import defaultdict

pd.set_option('max_rows', 10)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', -1)
# pd.set_option('display.width', 150)

In [2]:
# import warnings
# # warnings.filterwarnings("ignore")

In [3]:
# # Change format of charts to .svg
%config InlineBackend.figure_format = 'svg'

In [4]:
%xmode Plain

Exception reporting mode: Plain


In [5]:
df = pd.read_csv('telco_dataset.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [7]:
# Check for unique record ID #s - all were unique
df['customerID'].nunique()

7043

In [8]:
# Drop this column since we no longer need it
df.drop('customerID', axis=1, inplace=True)

In [9]:
copies = df.duplicated(subset=['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'])

In [10]:
copies.value_counts()

False    7021
True     22  
dtype: int64

I subsequently explored those 22 'duplicates' but determined they were all due to having the same characteristics -
people in their first month of payments who had the same services and had paid the same amount.

In [11]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [12]:
# Look at data to see if we need to recode any variables
df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [13]:
# Coded binary categories as 0 and 1 in preparation for regression
binary_dict = {'No': 0, 'No internet service': 0, 'No phone service': 0, 'Yes': 1}

df['Partner'] = df['Partner'].map(binary_dict)
df['Dependents'] = df['Dependents'].map(binary_dict)
df['PhoneService'] = df['PhoneService'].map(binary_dict)
df['MultipleLines'] = df['MultipleLines'].map(binary_dict)
df['OnlineSecurity'] = df['OnlineSecurity'].map(binary_dict)
df['OnlineBackup'] = df['OnlineBackup'].map(binary_dict)
df['DeviceProtection'] = df['DeviceProtection'].map(binary_dict)
df['TechSupport'] = df['TechSupport'].map(binary_dict)
df['StreamingTV'] = df['StreamingTV'].map(binary_dict)
df['StreamingMovies'] = df['StreamingMovies'].map(binary_dict)
df['PaperlessBilling'] = df['PaperlessBilling'].map(binary_dict)
df['Churn'] = df['Churn'].map(binary_dict)

In [14]:
# Code gender as binary
gender_dict = {'Male': 0, 'Female': 1}
df['gender'] = df['gender'].map(gender_dict)

# Rename column to 'female' since 1's now represent females
df.rename(columns={'gender':'female'})

,female,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,0,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,0,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,0,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,1,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,1,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0
7039,1,0,1,1,72,1,1,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0
7040,1,0,1,1,11,0,0,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,0,1,1,0,4,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1


In [15]:
# Confirm that our column renamed correctly - it did
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [16]:
# Recast binary columns as integers, as some of them were floats
df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies']].astype('int', inplace=True)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,0,0,0
2,1,1,0,0,0,0
3,1,0,1,1,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
7038,1,0,1,1,1,1
7039,0,1,1,0,1,1
7040,1,0,0,0,0,0
7041,0,0,0,0,0,0


Tried recasting TotalCharges to float to divide it by tenure, but got an error as there are 11 values in the column showing as ' '. This occurred because some people had not had any charges yet. I then dropped those records (only 11 of them) in order to allow us to cast the column as float.

In [17]:
# Dropping the 11 ' ' values from TotalCharges column
df = df[df['TotalCharges']!=' ']

# Then recast the column as float
df['TotalCharges'] = df['TotalCharges'].astype('float')

# Created new feature column to measure average monthly charges
df['AvgMonthlyCharges'] = df['TotalCharges']/df['tenure']

In [18]:
# Create dummy variables in preparation for regression.
# Drop first binary column for dummy variables to prevent redundant columns.
test = pd.get_dummies(data=df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)

In [31]:
# Change name of columnns for both dataframes to make them more easier to read and work with
df.columns = ['gender', 'senior', 'partner', 'dependents', 'tenure',
'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup',
'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
'contract', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'churn',
'avg_monthly_charges']

test.columns = ['gender', 'senior', 'partner', 'dependents', 'tenure',
'phone_service', 'multiple_lines', 'online_security', 'online_backup',
'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
'avg_monthly_charges', 'internet_service-fiber_optic',
'internet_service-no', 'contract-one_year', 'contract-two_year',
'payment_method-credit_card_auto',
'payment_method-electronic_check', 'payment_method-mailed_check']

Saving clean data and clean data + dummy variables for regression. Commenting code out below so we don't overwrite our current data files unless we mean to.

In [34]:
# # Save output as CSV
# df.to_csv('data/clean_data.csv', index=False)
# test.to_csv('data/clean_data_encoded_for_regression.csv', index=False)

### Feature engineering

In [5]:
df = pd.read_csv('data/clean_data.csv')

In [8]:
df.head()

,gender,senior,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn,avg_monthly_charges
0,1,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,29.850000
1,0,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0,55.573529
2,0,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,54.075000
3,0,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,40.905556
4,1,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,75.825000


In [12]:
df['monthly_charges'].describe()

count    7032.000000
mean     64.798208  
std      30.085974  
min      18.250000  
25%      35.587500  
50%      70.350000  
75%      89.862500  
max      118.750000 
Name: monthly_charges, dtype: float64

In [10]:
df['tenure'].describe()

count    7032.000000
mean     32.421786  
std      24.545260  
min      1.000000   
25%      9.000000   
50%      29.000000  
75%      55.000000  
max      72.000000  
Name: tenure, dtype: float64

In [11]:
df['internet_service'].value_counts()

Fiber optic    3096
DSL            2416
No             1520
Name: internet_service, dtype: int64

In [14]:
df.groupby(df['internet_service'])['monthly_charges'].mean()

internet_service
DSL            58.088017
Fiber optic    91.500129
No             21.076283
Name: monthly_charges, dtype: float64

In [22]:
x = df.groupby(df['tenure'])['monthly_charges'].mean()

In [25]:
df.groupby(df['phone_service'])['monthly_charges'].mean()

phone_service
0    41.992500
1    67.239625
Name: monthly_charges, dtype: float64

In [26]:
from sklearn.linear_model import LinearRegression

In [27]:
df.columns

Index(['gender', 'senior', 'partner', 'dependents', 'tenure', 'phone_service',
       'multiple_lines', 'internet_service', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'contract', 'paperless_billing', 'payment_method',
       'monthly_charges', 'total_charges', 'churn', 'avg_monthly_charges'],
      dtype='object')

In [28]:
X = df[['phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'internet_service-fiber_optic',
       'internet_service-no']]

y = df['monthly_charges']

SyntaxError: EOL while scanning string literal (<ipython-input-28-04d0430dc238>, line 4)